<a href="https://colab.research.google.com/github/linkyouhj/seq2seq/blob/main/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [72]:
from google.colab import drive 
drive.mount('/content/gdrive/')
path = '/content/gdrive/My Drive/Colab Notebooks/wmt16/'

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [73]:
def wmt_dataset(directory=path):
  ret = []
  splits = ['train','val','test']

  for filename in splits:
    examples = []

    en_path = os.path.join(directory, filename + '.en')
    de_path = os.path.join(directory, filename + '.de')
    en_file = [l.strip() for l in open(en_path, 'r', encoding='utf-8')]
    de_file = [l.strip() for l in open(de_path, 'r', encoding='utf-8')]
    assert len(en_file) == len(de_file)
    for i in range(len(en_file)):
      if en_file[i] != '' and de_file[i] != '' and en_file[i] != ' ' and de_file[i] != ' ' :
        examples.append({'en': en_file[i], 'de': de_file[i]})

    ret.append(examples)

  if len(ret) == 1:
    return ret[0]
  else:
    return tuple(ret)

In [74]:
dataset = wmt_dataset()

In [75]:
len(dataset)

3

In [76]:
dataset[0][0]['de']

'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.'

In [77]:
data_en = []
data_de = []
test_en = []
test_de = []
for i in range(2):
  for sentence in dataset[i]:
    data_en.append(sentence['en'])
    data_de.append(sentence['de'])

for sentence in dataset[2]:
  test_en.append(sentence['en'])
  test_de.append(sentence['en'])

In [78]:
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [79]:
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_en = []
for string in data_en:
    string = re.sub(r"[^a-zA-Z0-9]+", r" ", string)
    tokens = string.lower().strip()
    normalized_en.append(tokens)


# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_de = []
for string in data_de:
    string = re.sub(r"[^a-zA-ZÄäÖöÜüß0-9]+", r" ", string)
    tokens = string.lower().strip()
    normalized_de.append(tokens)

In [80]:
normalized_test_en = []
for string in test_en:
    string = re.sub(r"[^a-zA-Z0-9]+", r" ", string)
    tokens = string.lower().strip()
    normalized_test_en.append(tokens)


# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_test_de = []
for string in test_de:
    string = re.sub(r"[^a-zA-ZÄäÖöÜüß0-9]+", r" ", string)
    tokens = string.lower().strip()
    normalized_test_de.append(tokens)

In [81]:
result_de =  [word_tokenize(sentence) for sentence in normalized_de]
result_en =  [word_tokenize(sentence) for sentence in normalized_en]

In [82]:
result_test_de = [word_tokenize(sentence) for sentence in normalized_test_de]
result_test_en = [word_tokenize(sentence) for sentence in normalized_test_de]

In [83]:
from gensim.models import Word2Vec
model_en = Word2Vec(sentences=result_en, size = 256, window = 5,min_count=0,workers=4,sg=0)
model_de = Word2Vec(sentences=result_de, size = 256, window = 5,min_count=0,workers=4,sg=0)

In [84]:
from gensim.models import KeyedVectors
model_en.wv.save_word2vec_format('en_w2v') # 모델 저장
loaded_model_en = KeyedVectors.load_word2vec_format("en_w2v") # 모델 로드
model_de.wv.save_word2vec_format('de_w2v') # 모델 저장
loaded_model_de = KeyedVectors.load_word2vec_format("de_w2v") # 모델 로드

In [85]:
weights_en = torch.FloatTensor(loaded_model_en.vectors)
weights_de = torch.FloatTensor(loaded_model_de.vectors)

In [86]:
'''
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
'''
# 소문자, 다듬기, 그리고 문자가 아닌 문자 제거


def normalizeString(data):
    data["en"] = re.sub(r"[^a-zA-Z0-9]+", r" ", data["en"])
    data["en"] = data["en"].lower().strip()
    
    data["de"] = re.sub(r"[^a-zA-ZÄäÖöÜüß0-9]+", r" ", data["de"])
    data["de"] = data["de"].lower().strip()
    return data

In [87]:
MAX_LENGTH = 20

def filterPair(p,lang1,lang2):
    return 1<len(p[lang1].split(' ')) < MAX_LENGTH and \
        1<len(p[lang2].split(' ')) < MAX_LENGTH

def filterPairs(pairs,lang1,lang2):
    return [pair for pair in pairs if filterPair(pair,lang1,lang2)]

In [88]:
def readLangs(dataset, reverse=False):
    print("Reading lines...")

    # 모든 줄을 쌍으로 분리하고 정규화
    # for i in range(len(dataset)):
    #   for data in dataset[i]:
    #     data = normalizeString(data)
    data = []
    if len(dataset)>3:
      for d in dataset:
        data.append(d)
    else:
      for i in range(len(dataset)):
        for d in dataset[i]:
          data.append(d)
    for d in data:
      d = normalizeString(d)
    pairs = data

    # 쌍을 뒤집고, Lang 인스턴스 생성
    # input_lang = Lang(lang1)
    # output_lang = Lang(lang2)
    
    return pairs

In [89]:
def prepareData(lang1, lang2, dataset,reverse=False):
    pairs = readLangs(dataset, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs,lang1,lang2)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")

    return pairs


pairs = prepareData('de', 'en',dataset[:2], True)
print(random.choice(pairs))
test_pairs = prepareData('de', 'en',dataset[2], True)

Reading lines...
Read 30014 sentence pairs
Trimmed to 28450 sentence pairs
Counting words...
{'en': 'a lady with sunglasses on sitting on the top of a mountain beside of someone else', 'de': 'eine dame mit sonnenbrille sitzt auf der spitze eines berges neben einer anderen person'}
Reading lines...
Read 1000 sentence pairs
Trimmed to 948 sentence pairs
Counting words...


In [90]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding.from_pretrained(weights_de)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


In [91]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding.from_pretrained(weights_en)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [92]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# Transformer Model 적용


In [93]:
def indexesFromSentence(lang, sentence):
  if lang == 'en':
    try:
      return [loaded_model_en.vocab[word].index for word in sentence.lower().split(' ')]
    except KeyError:
      return []
    
  elif lang == 'de':
    try:
      return [loaded_model_de.vocab[word].index for word in sentence.lower().split(' ')]
    except KeyError:
      return []
  else: print("lang key error")

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair,lang1,lang2):
    input_tensor = tensorFromSentence(lang1, pair[lang1])
    target_tensor = tensorFromSentence(lang2, pair[lang2])
    return (input_tensor, target_tensor)

In [94]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]
    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing 포함: target을 다음 입력으로 전달
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Teacher forcing 미포함: 자신의 예측을 다음 입력으로 사용
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # 입력으로 사용할 부분을 히스토리에서 분리

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [95]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [96]:
def trainIters(encoder, decoder,lang1,lang2, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # print_every 마다 초기화
    plot_loss_total = 0  # plot_every 마다 초기화

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs),lang1,lang2)
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)


In [97]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # 주기적인 간격에 이 locator가 tick을 설정
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [107]:
SOS_token = 0
EOS_token = 1
hidden_size = 256
encoder1 = EncoderRNN(len(loaded_model_en.wv.vocab), hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, len(loaded_model_de.wv.vocab), dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1,'de','en', 75000, print_every=3000)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


1m 6s (- 26m 39s) (3000 4%) 3.9568
2m 9s (- 24m 43s) (6000 8%) 3.5927
3m 12s (- 23m 31s) (9000 12%) 3.4117
4m 16s (- 22m 29s) (12000 16%) 3.3072
5m 22s (- 21m 28s) (15000 20%) 3.2459
6m 26s (- 20m 23s) (18000 24%) 3.1705
7m 31s (- 19m 20s) (21000 28%) 3.0951
8m 36s (- 18m 16s) (24000 32%) 3.0556
9m 41s (- 17m 13s) (27000 36%) 3.0246
10m 47s (- 16m 10s) (30000 40%) 2.9924
11m 53s (- 15m 8s) (33000 44%) 2.9733
12m 58s (- 14m 3s) (36000 48%) 2.9712
14m 3s (- 12m 58s) (39000 52%) 2.9208
15m 9s (- 11m 54s) (42000 56%) 2.8608
16m 14s (- 10m 49s) (45000 60%) 2.8402
17m 20s (- 9m 45s) (48000 64%) 2.9154
18m 26s (- 8m 40s) (51000 68%) 2.8145
19m 32s (- 7m 35s) (54000 72%) 2.7959
20m 37s (- 6m 30s) (57000 76%) 2.7825
21m 42s (- 5m 25s) (60000 80%) 2.7868
22m 49s (- 4m 20s) (63000 84%) 2.7949
23m 54s (- 3m 15s) (66000 88%) 2.7835
24m 59s (- 2m 10s) (69000 92%) 2.7037
26m 5s (- 1m 5s) (72000 96%) 2.7932
27m 12s (- 0m 0s) (75000 100%) 2.7816


In [110]:
import nltk.translate.bleu_score as bleu

def evaluate(encoder, decoder, sentence,target, max_length=MAX_LENGTH,input_lang='de',output_lang='en'):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(loaded_model_en.wv.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()
        bleu_score = bleu.sentence_bleu(decoded_words,target)
        return decoded_words, decoder_attentions[:di + 1], bleu_score

In [111]:
def evaluateRandomly(encoder, decoder,lang1,lang2, n=10):
    final_score = 0.0
    for i in range(len(test_pairs)):
        pair = test_pairs[i]
        output_words, attentions, bleu_score = evaluate(encoder, decoder, pair[lang1],pair[lang2])
        final_score += bleu_score
    final_score = final_score/len(test_pairs)
    print(final_score)
    return final_score

In [112]:
evaluateRandomly(encoder1, attn_decoder1,'de','en')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.16532553137496744


0.16532553137496744

In [102]:
import nltk.translate.bleu_score as bleu
candidate = 'It is a guide to action which ensures that the military always obeys the commands of the party'
references = [
    'It is a guide to action that ensures that the military will forever heed Party commands',
]
print(bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)),candidate.split()))

0.41180376356915777


In [103]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence, target)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()
        bleu_score = bleu.sentence_bleu(decoded_words,target)
        return decoded_words, decoder_attentions[:di + 1], bleu_score

In [104]:
def evaluateRandomly(encoder, decoder,lang1,lang2, n=10):
    for i in range(len()):
        pair = random.choice(pairs)
        print('>', pair[lang1])
        print('=', pair[lang2])
        output_words, attentions = evaluate(encoder, decoder, pair[lang1],pair[lang2])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')